In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df_tmp = pd.merge(df3, df1, left_on='user_id', right_on='user_id')
df = pd.merge(df_tmp, df2, left_on='creative_id', right_on='creative_id')
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.head()

Using TensorFlow backend.


,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,30920,9,567330,1,504423,3,32638
1,320815,15,567330,1,504423,3,32638
2,355089,11,567330,1,504423,3,32638
3,363442,9,567330,1,504423,3,32638
4,370513,14,567330,1,504423,3,32638


In [2]:
df.sort_values(['user_id', 'time'], inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
16204330,1,20,877468,1,773445,5,29455
23067470,1,20,209778,1,188507,2,9702
26781033,1,20,821396,1,724607,5,7293
22502290,1,39,1683713,1,1458878,5,14668
9504689,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
28703035,900000,57,2692144,1,2316309,5,14372
15724163,900000,71,3391205,1,2918341,5,22137
7226547,900000,72,3471208,1,2989104,5,7283
9638586,900000,74,3527098,1,3037879,18,14681


In [3]:
df.reset_index(drop=True, inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,1,20,877468,1,773445,5,29455
1,1,20,209778,1,188507,2,9702
2,1,20,821396,1,724607,5,7293
3,1,39,1683713,1,1458878,5,14668
4,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
30082766,900000,57,2692144,1,2316309,5,14372
30082767,900000,71,3391205,1,2918341,5,22137
30082768,900000,72,3471208,1,2989104,5,7283
30082769,900000,74,3527098,1,3037879,18,14681


In [4]:
df['creative_id'].max()

4445718

In [5]:
x_tmp = np.array(df['creative_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(4445719, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['creative_id_0', 'creative_id_1', 'creative_id_2', 'creative_id_3', 'creative_id_4'])
X_tmp

(30082800, 1)
300828/300828 [==============================] - 3s 11us/step


,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4
0,-0.048975,0.043772,0.005149,-0.004764,0.045695
1,-0.019614,0.010676,0.046656,-0.013082,0.037023
2,-0.002463,-0.033769,0.016754,0.014599,0.021862
3,0.018736,-0.016554,-0.012494,-0.032890,0.049740
4,0.049470,0.041893,0.021906,0.038036,-0.044987
...,...,...,...,...,...
30082766,-0.007126,-0.015445,0.021976,-0.046910,-0.014177
30082767,0.022893,-0.044380,0.035772,-0.034094,-0.000664
30082768,0.026240,-0.009268,0.009897,-0.026745,-0.029143
30082769,-0.011952,0.018273,-0.006764,0.006807,-0.036429


In [6]:
X = df[['user_id', 'time', 'click_times']]

In [7]:
X = pd.concat([X, X_tmp], axis=1)
X.head()

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4
0,1,20,1,-0.048975,0.043772,0.005149,-0.004764,0.045695
1,1,20,1,-0.019614,0.010676,0.046656,-0.013082,0.037023
2,1,20,1,-0.002463,-0.033769,0.016754,0.014599,0.021862
3,1,39,1,0.018736,-0.016554,-0.012494,-0.032890,0.049740
4,1,40,1,0.049470,0.041893,0.021906,0.038036,-0.044987


In [8]:
df['ad_id'].max()

3812200

In [9]:
x_tmp = np.array(df['ad_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(3812201, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['ad_id_0', 'ad_id_1', 'ad_id_2', 'ad_id_3', 'ad_id_4'])
X_tmp

(30082800, 1)
300828/300828 [==============================] - 3s 10us/step


,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4
0,-0.033547,-0.036380,-0.044558,0.027335,0.036976
1,0.015520,0.035536,0.019043,0.038909,-0.020137
2,0.011261,0.007139,0.011110,-0.043634,-0.032178
3,-0.011223,0.036651,0.034127,0.048854,-0.008676
4,-0.040920,-0.004123,-0.009950,-0.011808,0.016795
...,...,...,...,...,...
30082766,-0.017851,0.033460,0.040128,0.014243,0.046341
30082767,0.045827,0.020064,0.040903,-0.033719,0.028912
30082768,0.001044,-0.024406,-0.009864,-0.007425,-0.049612
30082769,-0.029748,0.012752,0.010821,0.004813,0.034405


In [10]:
X = pd.concat([X, X_tmp], axis=1)
X.head()

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4
0,1,20,1,-0.048975,0.043772,0.005149,-0.004764,0.045695,-0.033547,-0.036380,-0.044558,0.027335,0.036976
1,1,20,1,-0.019614,0.010676,0.046656,-0.013082,0.037023,0.015520,0.035536,0.019043,0.038909,-0.020137
2,1,20,1,-0.002463,-0.033769,0.016754,0.014599,0.021862,0.011261,0.007139,0.011110,-0.043634,-0.032178
3,1,39,1,0.018736,-0.016554,-0.012494,-0.032890,0.049740,-0.011223,0.036651,0.034127,0.048854,-0.008676
4,1,40,1,0.049470,0.041893,0.021906,0.038036,-0.044987,-0.040920,-0.004123,-0.009950,-0.011808,0.016795


In [11]:
df['product_category'].max()

18

In [12]:
x_tmp = np.array(df['product_category'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(20, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['product_category_0', 'product_category_1', 
                                                   'product_category_2', 'product_category_3', 'product_category_4'])
X = pd.concat([X, X_tmp], axis=1)
X.head()

(30082800, 1)
300828/300828 [==============================] - 2s 7us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4
0,1,20,1,-0.048975,0.043772,0.005149,-0.004764,0.045695,-0.033547,-0.036380,-0.044558,0.027335,0.036976,-0.036599,-0.025282,-0.041565,-0.006554,0.025688
1,1,20,1,-0.019614,0.010676,0.046656,-0.013082,0.037023,0.015520,0.035536,0.019043,0.038909,-0.020137,-0.018892,-0.008490,0.020219,-0.024885,-0.005556
2,1,20,1,-0.002463,-0.033769,0.016754,0.014599,0.021862,0.011261,0.007139,0.011110,-0.043634,-0.032178,-0.036599,-0.025282,-0.041565,-0.006554,0.025688
3,1,39,1,0.018736,-0.016554,-0.012494,-0.032890,0.049740,-0.011223,0.036651,0.034127,0.048854,-0.008676,-0.036599,-0.025282,-0.041565,-0.006554,0.025688
4,1,40,1,0.049470,0.041893,0.021906,0.038036,-0.044987,-0.040920,-0.004123,-0.009950,-0.011808,0.016795,-0.018892,-0.008490,0.020219,-0.024885,-0.005556


In [13]:
df['advertiser_id'].max()

62965

In [14]:
x_tmp = np.array(df['advertiser_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(62966, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['advertiser_id_0', 'advertiser_id_1', 
                                                   'advertiser_id_2', 'advertiser_id_3', 'advertiser_id_4'])
X = pd.concat([X, X_tmp], axis=1)
X.head()

300828/300828 [==============================] - 2s 8us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,1,20,1,-0.048975,0.043772,0.005149,-0.004764,0.045695,-0.033547,-0.036380,...,-0.036599,-0.025282,-0.041565,-0.006554,0.025688,-0.046636,-0.009082,0.042505,0.005330,0.022218
1,1,20,1,-0.019614,0.010676,0.046656,-0.013082,0.037023,0.015520,0.035536,...,-0.018892,-0.008490,0.020219,-0.024885,-0.005556,0.006839,0.027888,0.003329,-0.030744,0.027253
2,1,20,1,-0.002463,-0.033769,0.016754,0.014599,0.021862,0.011261,0.007139,...,-0.036599,-0.025282,-0.041565,-0.006554,0.025688,-0.000538,0.028795,-0.014415,-0.015796,-0.021657
3,1,39,1,0.018736,-0.016554,-0.012494,-0.032890,0.049740,-0.011223,0.036651,...,-0.036599,-0.025282,-0.041565,-0.006554,0.025688,-0.037419,-0.049961,-0.002889,0.019592,-0.045036
4,1,40,1,0.049470,0.041893,0.021906,0.038036,-0.044987,-0.040920,-0.004123,...,-0.018892,-0.008490,0.020219,-0.024885,-0.005556,-0.039228,-0.012258,0.033792,0.044499,0.018611


In [15]:
def check_matrix(x):
    if x.shape[0] >= 30:
        return x.iloc[:30, :]
    else:
        tmp1 = np.zeros((30 - x.shape[0], x.shape[1]))
        tmp1 = pd.DataFrame(tmp1)
        tmp1.columns = x.columns
        return pd.concat([x, tmp1], axis=0)
X = X.groupby('user_id').apply(check_matrix)
X = X.reset_index(drop=True)
X = X.drop('user_id', axis = 1)
X

,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,20.0,1.0,-0.048975,0.043772,0.005149,-0.004764,0.045695,-0.033547,-0.036380,-0.044558,...,-0.036599,-0.025282,-0.041565,-0.006554,0.025688,-0.046636,-0.009082,0.042505,0.005330,0.022218
1,20.0,1.0,-0.019614,0.010676,0.046656,-0.013082,0.037023,0.015520,0.035536,0.019043,...,-0.018892,-0.008490,0.020219,-0.024885,-0.005556,0.006839,0.027888,0.003329,-0.030744,0.027253
2,20.0,1.0,-0.002463,-0.033769,0.016754,0.014599,0.021862,0.011261,0.007139,0.011110,...,-0.036599,-0.025282,-0.041565,-0.006554,0.025688,-0.000538,0.028795,-0.014415,-0.015796,-0.021657
3,39.0,1.0,0.018736,-0.016554,-0.012494,-0.032890,0.049740,-0.011223,0.036651,0.034127,...,-0.036599,-0.025282,-0.041565,-0.006554,0.025688,-0.037419,-0.049961,-0.002889,0.019592,-0.045036
4,40.0,1.0,0.049470,0.041893,0.021906,0.038036,-0.044987,-0.040920,-0.004123,-0.009950,...,-0.018892,-0.008490,0.020219,-0.024885,-0.005556,-0.039228,-0.012258,0.033792,0.044499,0.018611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26999995,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999996,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999997,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999998,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
# X.to_csv('data/train_preliminary/lstm_5.csv')

In [17]:
X = preprocessing.MinMaxScaler().fit_transform(X)


In [18]:
X = np.array(X).reshape((900000, 30, 22))

In [28]:
from keras.layers.core import Masking
X_train, X_test = X[:800000, :, :], X[800000:, :, :]
Y2_train, Y2_test = df1.iloc[:800000, 2:3], df1.iloc[800000:, 2:3]
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(30, 22)))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam')
model.fit(X_train, Y2_train, nb_epoch=20)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/20
800000/800000 [==============================] - 469s 587us/step - loss: 0.2106
Epoch 2/20
800000/800000 [==============================] - 470s 587us/step - loss: 0.1858
Epoch 3/20
800000/800000 [==============================] - 469s 587us/step - loss: 0.1734
Epoch 4/20
800000/800000 [==============================] - 482s 603us/step - loss: 0.1658
Epoch 5/20
800000/800000 [==============================] - 496s 620us/step - loss: 0.1602
Epoch 6/20
800000/800000 [==============================] - 492s 616us/step - loss: 0.1556
Epoch 7/20
800000/800000 [==============================] - 489s 611us/step - loss: 0.1518
Epoch 8/20
800000/800000 [==============================] - 496s 620us/step - loss: 0.1489
Epoch 9/20
800000/800000 [==============================] - 498s 622us/step - loss: 0.1465
Epoch 10/20
800000/800000 [==============================] - 491s 613us/step - loss: 0.1446
Epoch 11/20
800000/800000 [==============================] - 482s 602us/step - loss: 0.14

In [29]:
Y2_train.shape

(800000, 1)

In [31]:
Y = model.predict(X_train)
Y = np.round(Y)
ans = 0
for i in range(800000):
    if Y[i] == Y2_train.iloc[i, 0]:
        ans += 1
print('accurency:', ans/800000)

accurency: 0.808205


In [32]:
Y = model.predict(X_test)
Y = np.round(Y)
ans = 0
for i in range(100000):
    if Y[i] == Y2_test.iloc[i, 0]:
        ans += 1
print('accurency:', ans/100000)

accurency: 0.80583
